In [1]:
import numpy as np
import matplotlib.pyplot as plt

import random

from matplotlib.colors import ListedColormap
from sklearn import datasets, model_selection

#### Задание 1 
В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.).

#### Задание 2 
Для задачи классификации обучить дерево решений с использованием критериев разбиения Джини и Энтропия. Сравнить качество классификации, сделать выводы.

In [2]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [3]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()        
        
    def predict(self):
              
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)        
        return prediction    

In [4]:
# Расчет критерия Джини, энтропии, дисперсии
def info_criteria(labels, criteria):
    if criteria=='variance': 
        return np.mean((labels - np.mean(labels))**2) 
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    entropy_ = 0
    impurity = 1    
    for label in classes:
        p = classes[label] / len(labels)
        entropy_ -= p * np.log2(p)
        impurity -= p ** 2    
        
    if criteria == 'entropy':        
        return entropy_
    elif criteria == 'gini':
        return impurity   

In [5]:
# Расчет качества

def quality(left_labels, right_labels, current_criteria, criteria):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_criteria - p * info_criteria(left_labels, criteria) - (1 - p) *info_criteria(right_labels, criteria)

In [6]:
# Разбиение датасета в узле

def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [7]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels, n_features, min_leaf, criteria):

    current_criteria = info_criteria(labels, criteria)

    best_quality = 0
    best_t = None
    best_index = None
    
    #n_features = data.shape[1]
    random.seed(42)
    selected_features =  random.sample(range(data.shape[1]), n_features)
    
    for index in selected_features:
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique([row[index] for row in data])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_criteria, criteria)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [8]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels, n_features, min_leaf, criteria): 

    quality, t, index = find_best_split(data, labels, n_features, min_leaf, criteria)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, n_features, min_leaf, criteria)
    false_branch = build_tree(false_data, false_labels, n_features, min_leaf, criteria)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [9]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [10]:
def predict(data, tree):    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)        
        classes.append(prediction)
    return classes

#### Генерируем данные

In [11]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_classification(n_features = 12, n_informative = 9, 
                                                      n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=42)

In [12]:
# Разобьем выборку на обучающую и тестовую
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

####  n_features=8, min_leaf=5, criteria='entropy'

In [13]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_labels, n_features=8, min_leaf=5, criteria='entropy') #gini 

In [14]:
# Получим ответы для обучающей выборки 
train_answers = predict(train_data, my_tree)

In [15]:
# И получим ответы для тестовой выборки
answers = predict(test_data, my_tree)

In [16]:
# Введем функцию подсчета точности как доли правильных ответов
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [17]:
# Точность на обучающей выборке
train_accuracy = accuracy_metric(train_labels, train_answers)
train_accuracy

94.28571428571428

In [18]:
# Точность на тестовой выборке
test_accuracy = accuracy_metric(test_labels, answers)
test_accuracy

93.33333333333333

#### Выводы:

Критерии разбиения Джини и Энтропия дают одинаковое (или очень близкое) качество классификации.

#### Задание 3*
Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [19]:
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()        
        
    def predict(self):              
        prediction = np.mean(self.labels)        
        return prediction    

In [20]:
X, y = datasets.make_regression(n_samples=1000, n_features=15, random_state=42)
# Разобьем выборку на обучающую и тестовую
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    test_size = 0.3,
                                                                    random_state = 1)

In [21]:
# Построим дерево по обучающей выборке
my_tree = build_tree(X_train, y_train, n_features=12, min_leaf=7, criteria='variance') 

In [22]:
# Получим ответы для обучающей выборки 
y_pred_train = predict(X_train, my_tree)

# И получим ответы для тестовой выборки
y_pred_test = predict(X_test, my_tree)

In [23]:
def R2(actual, predicted):
    SS_total = ((actual-np.mean(actual))**2).sum()
    SS_res = ((actual - predicted)**2).sum()
    return 1 - SS_res / SS_total

In [24]:
# R2 на обучающей выборке
R2_train = R2(y_train, y_pred_train)
R2_train

0.8383772939013412

In [25]:
# R2 на тестовой выборке
R2_test = R2(y_test, y_pred_test)
R2_test

0.6482068587499689